In [1]:
import json
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
import numpy as np
from glob import glob 
import re 
from nltk import word_tokenize as lib_tokenizer 
import math

In [2]:
df = pd.read_csv("../data/processed/wikipedia_20220620_cleaned_v2.csv")
train = json.load(open("../data/raw/e2eqa-train+public_test-v1/zac2022_train_merged_final.json"))

In [3]:
for x in train['data']:
    x['dirty_text'] = None

In [4]:
all_titles = dict([(x.strip(),True) for x in open("../data/raw/wikipedia_20220620_cleaned/wikipedia_20220620_all_titles.txt").readlines()])
all_titles

{'Trang Chính': True,
 'Internet Society': True,
 'Tiếng Việt': True,
 'Ohio': True,
 'California': True,
 'Thụy Điển': True,
 'Thành phố Hồ Chí Minh': True,
 'Lào Cai': True,
 'W3C': True,
 'Bộ Kế hoạch và Đầu tư (Việt Nam)': True,
 'Lào': True,
 'Hoa Kỳ': True,
 'Hà Giang': True,
 'Cao Bằng': True,
 'Iraq': True,
 'Hà Nội': True,
 'Campuchia': True,
 'VIQR': True,
 'Việt Nam Cộng hòa': True,
 'Sacramento, California': True,
 'Los Angeles': True,
 'San Francisco': True,
 'San Diego': True,
 'Người Mỹ gốc Việt': True,
 'Giấy phép Tài liệu Tự do GNU': True,
 'Lý Thường Kiệt': True,
 'Quang Trung': True,
 'Hồ Biểu Chánh': True,
 'Bắc Kạn': True,
 'Lạng Sơn': True,
 'A': True,
 'B': True,
 'C': True,
 'D': True,
 'E': True,
 'F': True,
 'G': True,
 'H': True,
 'I': True,
 'J': True,
 'K': True,
 'L': True,
 'M': True,
 'N': True,
 'O': True,
 'P': True,
 'Q': True,
 'R': True,
 'S': True,
 'T': True,
 'U': True,
 'V': True,
 'W': True,
 'X': True,
 'Y': True,
 'Z': True,
 'Tuyên Quang': T

In [5]:
dict_map = dict({}) 
 
def word_tokenize(text): 
    global dict_map 
    words = text.split() 
    words_norm = [] 
    for w in words: 
        if dict_map.get(w, None) is None: 
            dict_map[w] = ' '.join(lib_tokenizer(w)).replace('``', '"').replace("''", '"') 
        words_norm.append(dict_map[w]) 
    return words_norm 
 
def strip_answer_string(text): 
    text = text.strip() 
    while text[-1] in '.,/><;:\'"[]{}+=-_)(*&^!~`': 
        if text[0] != '(' and text[-1] == ')' and '(' in text: 
            break 
        if text[-1] == '"' and text[0] != '"' and text.count('"') > 1: 
            break 
        text = text[:-1].strip() 
    while text[0] in '.,/><;:\'"[]{}+=-_)(*&^!~`': 
        if text[0] == '"' and text[-1] != '"' and text.count('"') > 1: 
            break 
        text = text[1:].strip() 
    text = text.strip() 
    return text 
 
def strip_context(text): 
    text = text.replace('\n', ' ') 
    text = re.sub(r'\s+', ' ', text) 
    text = text.strip() 
    return text

In [6]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, use_memory_fs=False, nb_workers=6)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [7]:
def apply_tokenizer(df, num_chunks):
    chunk_sr = pd.Series()
    i = 0
    end = df.shape[0]
    chunk_size = math.floor(end/num_chunks)
    while i != end:
        if end - i < chunk_size:
            chunk_size = end - i
        temp_df = df[i:i+chunk_size].copy()
        chunk_sr = pd.concat([chunk_sr, temp_df["text"].parallel_apply(lambda x: " ".join(word_tokenize(strip_context(x))))], ignore_index=True)
        i+=chunk_size
    return chunk_sr

df["cleaned_text"] = apply_tokenizer(df, 9)
df["cleaned_text"].head()

0    Trang Chính < templatestyles src= " Wiki2021/s...
1    Internet Society hay ISOC là một tổ chức quốc ...
2    Tiếng Việt , cũng gọi là tiếng Việt Nam hay Vi...
3    hệ thống thanh điệu phát triển cao hơn , hệ th...
4    tiếp xúc Hán – Việt thành 2 giai đoạn chính : ...
Name: cleaned_text, dtype: object

In [8]:
train_titles = set([x['title'].strip() for x in train['data'] if len(x['title']) > 0])

In [9]:
df["valid"] = df['title'].parallel_apply(lambda x: str(x).strip() in train_titles)

In [10]:
df = df[df['valid']]
df.reset_index(drop=True, inplace=True)

In [11]:
df.title.unique().shape

(8782,)

In [12]:
df.head()

,title,text,bm25_text,cleaned_text,valid
0,Tiếng Việt,"Tiếng Việt , cũng gọi là tiếng Việt Nam hay Vi...",tiếng việt cũng gọi là tiếng việt nam hay việt...,"Tiếng Việt , cũng gọi là tiếng Việt Nam hay Vi...",True
1,Tiếng Việt,"hệ thống thanh điệu phát triển cao hơn, hệ thố...",hệ thống thanh điệu phát triển cao hơn hệ thốn...,"hệ thống thanh điệu phát triển cao hơn , hệ th...",True
2,Tiếng Việt,tiếp xúc Hán – Việt thành 2 giai đoạn chính: \...,tiếp xúc hán – việt thành 2 giai đoạn chính bu...,tiếp xúc Hán – Việt thành 2 giai đoạn chính : ...,True
3,Tiếng Việt,"thêm hàng loạt các yếu tố Hán–Việt. Như là ""ch...",thêm hàng loạt các yếu tố hán–việt như là chủ ...,"thêm hàng loạt các yếu tố Hán–Việt . Như là "" ...",True
4,Tiếng Việt,tiếng Hán vẫn có ai đó chấp nhận và sử dụng tr...,tiếng hán vẫn có ai đó chấp nhận và sử dụng tr...,tiếng Hán vẫn có ai đó chấp nhận và sử dụng tr...,True


In [13]:
train_df = pd.DataFrame()
train_df["id"] = [x['id'] for x in train['data']]
train_df["title"] = [x['title'] for x in train['data']]
train_df["text"] = [x['text'] for x in train['data']]

In [16]:
train_df

,id,title,text
0,718d41cd997b2b44b0685ac54aa55bd8,Trung Quốc,Thủ tướng Trung Quốc là nhân vật lãnh đạo chín...
1,c926e7b0717202618a10dd907d4b4c39,,"có 23 quốc gia không có lực lượng quân đội, ba..."
2,d38ef5bf1fb82b410026ed82c8a44cae,Raymondienne,Raymondienne (hay Raymonde Dien) sinh ngày 13 ...
3,b6b5589a98fdccd208dc752bac853993,Cúp cờ vua thế giới,Cúp cờ vua thế giới là tên gọi một số giải đấu...
4,82396a18fa9812bfec4d3ecb7ae60905,Shkhara,Đỉnh núi nằm ở phần trung tâm của dãy núi Đại ...
...,...,...,...
20852,508022f540c39fe31511f594748759bc,Eros,"Trong thần thoại Hy Lạp , "" Eros "" là vị thần ..."
20853,93c746695c50932ac45ac498a192a3e5,Lịch sử hành chính Hà Nội,"Vào thời điểm hiện tại ( 2017 ) , về mặt hành ..."
20854,c477d4b40045ee4251cf9b2a0482cfc4,Nhật ký trong tù,"“ Nhật ký trong tù ” là một cuốn sổ tay nhỏ , ..."
20855,278ad127825c085a54fa22116c281f92,Google,Tên miền www.google.com được đăng ký ngày 15 t...


In [50]:
len(train_titles)
'Bánh gai Tứ Trụ' in list(train_titles - set(df['title'].unique())) 

True

Nếu `title` của `train_df` không nằm trong `df` thì đó là cleaned `title` (nghĩa là chỉ có một kết quả), ngược lại nếu nằm trong `df` thì là dirty (có nhiều ứng viên cho kết quả dựa vào thuật toán ranked bm25). 

In [51]:
def find_dirty_text(title, text):
    text= " ".join(word_tokenize(strip_context(text)))
    if text[-1] == ".":
        text = text[:-1].strip()
    tmp = df[df.title == title].reset_index(drop=True)
    if len(tmp) == 0:
#         print(title)
        return None
    for candidate in tmp.cleaned_text:
        if text in candidate:
#             print("Found dirty")
            return candidate
    return text

In [52]:
train_df["dirty_text"] = train_df.parallel_apply(lambda row: find_dirty_text(row.title, row.text),axis=1)

In [53]:
train_df

,id,title,text,dirty_text
0,718d41cd997b2b44b0685ac54aa55bd8,Trung Quốc,Thủ tướng Trung Quốc là nhân vật lãnh đạo chín...,Thủ tướng Trung Quốc là nhân vật lãnh đạo chín...
1,c926e7b0717202618a10dd907d4b4c39,,"có 23 quốc gia không có lực lượng quân đội, ba...",None
2,d38ef5bf1fb82b410026ed82c8a44cae,Raymondienne,Raymondienne (hay Raymonde Dien) sinh ngày 13 ...,None
3,b6b5589a98fdccd208dc752bac853993,Cúp cờ vua thế giới,Cúp cờ vua thế giới là tên gọi một số giải đấu...,Cúp cờ vua thế giới là tên gọi một số giải đấu...
4,82396a18fa9812bfec4d3ecb7ae60905,Shkhara,Đỉnh núi nằm ở phần trung tâm của dãy núi Đại ...,Shkhara ( ) là núi cao thứ ba trong dãy núi Ka...
...,...,...,...,...
20852,508022f540c39fe31511f594748759bc,Eros,"Trong thần thoại Hy Lạp , "" Eros "" là vị thần ...","Eros Trong thần thoại Hy Lạp , "" Eros "" là vị ..."
20853,93c746695c50932ac45ac498a192a3e5,Lịch sử hành chính Hà Nội,"Vào thời điểm hiện tại ( 2017 ) , về mặt hành ...","Vào thời điểm hiện tại ( 2017 ) , về mặt hành ..."
20854,c477d4b40045ee4251cf9b2a0482cfc4,Nhật ký trong tù,"“ Nhật ký trong tù ” là một cuốn sổ tay nhỏ , ...","“ Nhật ký trong tù ” là một cuốn sổ tay nhỏ , ..."
20855,278ad127825c085a54fa22116c281f92,Google,Tên miền www.google.com được đăng ký ngày 15 t...,Tên miền www.google.com được đăng ký ngày 15 t...


In [54]:
train_df[train_df.dirty_text.isna()]

,id,title,text,dirty_text
1,c926e7b0717202618a10dd907d4b4c39,,"có 23 quốc gia không có lực lượng quân đội, ba...",None
2,d38ef5bf1fb82b410026ed82c8a44cae,Raymondienne,Raymondienne (hay Raymonde Dien) sinh ngày 13 ...,None
14,361a30769c1a5dca2a7b8f5c7f601982,Bánh gai Tứ Trụ,"Làng Mía thuộc hữu ngạn sông Chu, cách thị trấ...",None
18,5d7d3b0d5cd7b2917a2081c3b4d7c8a8,Nhà nước Hồi giáo ( chính thể ),Khái niệm về nhà nước Hồi giáo hiện đại đã đượ...,None
20,d7000d411d3f04dee7fa327ef11e3db0,Quan hệ Israel – Liban,Thủ tướng Liban Fouad Siniora cho biết vào thá...,None
...,...,...,...,...
20804,cb4d3b162a56b52a4927ab29b739c7a2,Thủy điện An Khê - Kanak,Nhà máy được khởi công xây dựng vào ngày 14 th...,None
20834,d701dfcba59420604c66e803c9556754,Sao Hoả,"Sao Hoả còn gọi là : Hoả Tinh , ( Tiếng Anh : ...",None
20836,5769cee0487a9674306d271854244b39,,Máy quay phim: Phát minh kỳ diệu của anh em Lu...,None
20843,5f420b6d68436b684f33dcded61d5a3c,Egil Kapstad,"Ông sinh ra tại Oslo, thủ đô của Na Uy và thàn...",None


In [55]:
dirty_text_dict = dict([x,y] for x,y in zip(train_df.id, train_df.dirty_text))

In [56]:
for x in train['data']:
    x['dirty_text'] = dirty_text_dict.get(x['id'],x['text'])

In [57]:
json.dump(train,open("../data/processed/zac2022_train_merged_final.json","wt"))